In [1]:
# Dependencies.
import pandas as pd
import numpy as np
import datetime as dt

# SQL.
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func


In [2]:
# Filepath to the CSV.
filepath = "resources/raw_border_crossing_data.csv"

In [3]:
# Load data into Pandas.
raw_df = pd.read_csv(filepath)

display(raw_df.info())
display(raw_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393329 entries, 0 to 393328
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Port Name  393329 non-null  object 
 1   State      393329 non-null  object 
 2   Port Code  393329 non-null  int64  
 3   Border     393329 non-null  object 
 4   Date       393329 non-null  object 
 5   Measure    393329 non-null  object 
 6   Value      393329 non-null  int64  
 7   Latitude   393329 non-null  float64
 8   Longitude  393329 non-null  float64
 9   Point      393329 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 30.0+ MB


None

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
0,Roma,Texas,2310,US-Mexico Border,Dec 2023,Buses,46,26.404,-99.019,POINT (-99.018981 26.403928)
1,Del Rio,Texas,2302,US-Mexico Border,Dec 2023,Trucks,6552,29.327,-100.928,POINT (-100.927612 29.326784)
2,Roma,Texas,2310,US-Mexico Border,Nov 2023,Trucks,3753,26.404,-99.019,POINT (-99.018981 26.403928)
3,Douglas,Arizona,2601,US-Mexico Border,Oct 2023,Buses,13,31.334,-109.560,POINT (-109.560344 31.334043)
4,Beecher Falls,Vermont,206,US-Canada Border,Aug 2023,Trucks,422,45.013,-71.505,POINT (-71.505309 45.013411)


### Data Cleaning
We have no missing values. That's great.  
We have some data type mismatching...  
Date should be split into Year and Month (months need to be transcribed to numbers).  
Column names should be formatted for easier use with code.  

In [4]:
# Make a copy of the DataFrame.
df = raw_df.copy()

In [5]:
# Change the column names to be lowercase and without spaces (change them to _).
df.columns = [x.lower().replace(' ', '_') for x in df.columns]

# Check.
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393329 entries, 0 to 393328
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   port_name  393329 non-null  object 
 1   state      393329 non-null  object 
 2   port_code  393329 non-null  int64  
 3   border     393329 non-null  object 
 4   date       393329 non-null  object 
 5   measure    393329 non-null  object 
 6   value      393329 non-null  int64  
 7   latitude   393329 non-null  float64
 8   longitude  393329 non-null  float64
 9   point      393329 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 30.0+ MB


None

,port_name,state,port_code,border,date,measure,value,latitude,longitude,point
0,Roma,Texas,2310,US-Mexico Border,Dec 2023,Buses,46,26.404,-99.019,POINT (-99.018981 26.403928)
1,Del Rio,Texas,2302,US-Mexico Border,Dec 2023,Trucks,6552,29.327,-100.928,POINT (-100.927612 29.326784)
2,Roma,Texas,2310,US-Mexico Border,Nov 2023,Trucks,3753,26.404,-99.019,POINT (-99.018981 26.403928)
3,Douglas,Arizona,2601,US-Mexico Border,Oct 2023,Buses,13,31.334,-109.560,POINT (-109.560344 31.334043)
4,Beecher Falls,Vermont,206,US-Canada Border,Aug 2023,Trucks,422,45.013,-71.505,POINT (-71.505309 45.013411)


In [6]:
# Change the date column to be a year and month column instead.
# Split date column into month and year columns.
df[['month','year']] = df.date.str.split(' ', expand=True)

# Change year column to Integer.
df['year'] = df['year'].astype(int)

# Transcribe and change month column to Integer.
months = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}

df['month'] = df['month'].map(months)
df['month'] = df['month'].astype(int)

# Drop the original date column and the point column.
df = df.drop(columns=['date', 'point'])

# Check.
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393329 entries, 0 to 393328
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   port_name  393329 non-null  object 
 1   state      393329 non-null  object 
 2   port_code  393329 non-null  int64  
 3   border     393329 non-null  object 
 4   measure    393329 non-null  object 
 5   value      393329 non-null  int64  
 6   latitude   393329 non-null  float64
 7   longitude  393329 non-null  float64
 8   month      393329 non-null  int32  
 9   year       393329 non-null  int32  
dtypes: float64(2), int32(2), int64(2), object(4)
memory usage: 27.0+ MB


None

,port_name,state,port_code,border,measure,value,latitude,longitude,month,year
0,Roma,Texas,2310,US-Mexico Border,Buses,46,26.404,-99.019,12,2023
1,Del Rio,Texas,2302,US-Mexico Border,Trucks,6552,29.327,-100.928,12,2023
2,Roma,Texas,2310,US-Mexico Border,Trucks,3753,26.404,-99.019,11,2023
3,Douglas,Arizona,2601,US-Mexico Border,Buses,13,31.334,-109.560,10,2023
4,Beecher Falls,Vermont,206,US-Canada Border,Trucks,422,45.013,-71.505,8,2023


### Data Filtering
We do not want to use the full range of data entries.  
Some of the entries are quite old and we don't think those will be needed.  
So, we will shave off the entries to a more acceptable range for us; 2018 until present.

In [7]:
# Find out where we stand in terms of years.
df.year.value_counts()

year
2010    16200
2005    16128
2006    16128
2009    16056
2007    16056
2008    16056
2004    15984
2015    15984
2013    15840
2012    15840
2011    15840
2014    15840
2003    15588
1999    14832
1998    14832
1997    14832
1996    14832
2002    14832
2001    14832
2000    14832
2016    13754
2017     9717
2019     9588
2018     9529
2023     9087
2022     8876
2020     8477
2021     8430
2024     4507
Name: count, dtype: int64

#### Our Range
Here, we'd like to give a decade look at the dataset.  
We need to cut it down to 2014 until 2024.  

In [8]:
# Filter the data.
# Define the filter.
filter = df.year >= 2014

# Apply the filter.
filter_df = df[filter]

# Check.
filter_df.year.describe()

count    113789.000000
mean       2018.099096
std           3.125717
min        2014.000000
25%        2015.000000
50%        2018.000000
75%        2021.000000
max        2024.000000
Name: year, dtype: float64

In [9]:
# Final look and export the clean CSV.
display(filter_df.info())
display(filter_df.head())

# Export.
filter_df.to_csv("resources/clean_border_crossing_data.csv", index = False)

<class 'pandas.core.frame.DataFrame'>
Index: 113789 entries, 0 to 393328
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   port_name  113789 non-null  object 
 1   state      113789 non-null  object 
 2   port_code  113789 non-null  int64  
 3   border     113789 non-null  object 
 4   measure    113789 non-null  object 
 5   value      113789 non-null  int64  
 6   latitude   113789 non-null  float64
 7   longitude  113789 non-null  float64
 8   month      113789 non-null  int32  
 9   year       113789 non-null  int32  
dtypes: float64(2), int32(2), int64(2), object(4)
memory usage: 8.7+ MB


None

,port_name,state,port_code,border,measure,value,latitude,longitude,month,year
0,Roma,Texas,2310,US-Mexico Border,Buses,46,26.404,-99.019,12,2023
1,Del Rio,Texas,2302,US-Mexico Border,Trucks,6552,29.327,-100.928,12,2023
2,Roma,Texas,2310,US-Mexico Border,Trucks,3753,26.404,-99.019,11,2023
3,Douglas,Arizona,2601,US-Mexico Border,Buses,13,31.334,-109.560,10,2023
4,Beecher Falls,Vermont,206,US-Canada Border,Trucks,422,45.013,-71.505,8,2023


### Database Creation
Now, we need to create the sqlite database for the project.  

In [10]:
# Create an engine to make the database.
engine = create_engine("sqlite:///border_database.sqlite")

In [11]:
# Write the DataFrame to the database.
filter_df.to_sql("crossings", con = engine, index = False, if_exists = "replace")

113789

In [19]:
# Inspect the database to see if the creation was successful.
inspector = inspect(engine)

# Get the tables.
tables = inspector.get_table_names()

# Print the metadata for each table.
for table in tables:
    print(table)
    print()

    columns = inspector.get_columns(table)

    for column in columns:
        print(column['name'], column['type'])

    print()

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [20]:
# It worked.
# Kill the engine.
engine.dispose()